In [1]:
from pymongo import MongoClient
client = MongoClient()
db = client.whosampled
import numpy as np
import pandas as pd
import re

import sys
np.set_printoptions(threshold=sys.maxsize)

from turn_db_main_into_utility_matrix import from_mongo_collection_to_utility_matrix

In [2]:
utility_to_song, utility_to_artist, df = from_mongo_collection_to_utility_matrix(db.main_redo)

In [3]:
# Problem
# How many times are we missing data on producers?
len(df[df.sampled_artist == "None Listed"])

#Very few?
#Only lists 100 out of 17,000 times there is no new_song_producer.
#Only 72 missing sampled_artist

#There is no problem with 'The', there's very few duplications.

0

In [4]:
# How sparse is matrix?

import scipy.sparse as sparse
sparse_util = sparse.lil_matrix(utility_to_song)

matrix_size = sparse_util.shape[0]*sparse_util.shape[1] # Number of possible interactions in the matrix
num_purchases = len(sparse_util.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

#We have a matrix that is 99.8% sparse. Kind of a problem. According to https://jessesw.com/Rec-System/ 
# maximum is 99.5%


99.87388711015949

In [26]:
# How sparse is matrix?

import scipy.sparse as sparse
sparse_artists_util = sparse.lil_matrix(utility_to_artist)

matrix_size = sparse_artists_util.shape[0]*sparse_util.shape[1] # Number of possible interactions in the matrix
num_purchases = len(sparse_artists_util.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

#We have a matrix that is 99.8% sparse. Kind of a problem. According to https://jessesw.com/Rec-System/ 
# maximum is 99.5%

#If we go by artist, we get down to 99.765%

99.76617989333165


There are 15809 unique records. There are 1324 producers. There are 7789 songs sampled.

In [28]:
#Using the github benfred implicit. Docstring says confidence matrix is important.

#I think it's necessary to put the matrix in the form of users as x axis, items as y-axis.
import os
os.environ["OPENBLAS_NUM_THREADS"]="1"

import implicit
model = implicit.als.AlternatingLeastSquares(calculate_training_loss = True, iterations=100, factors=50)

model.fit(sparse_artists_util)

100%|██████████| 100.0/100 [00:02<00:00, 34.07it/s, loss=0.000932]


In [11]:
user_items = sparse_util.tocsr()

In [25]:
recommendations = model.recommend(3000, user_items)

IndexError: index (3000) out of range

In [24]:
recommendations

[(355, 0.4986826),
 (691, 0.49190974),
 (707, 0.17330953),
 (602, 0.09364148),
 (666, 0.09328874),
 (336, 0.09155294),
 (1053, 0.08850966),
 (1360, 0.07718784),
 (1118, 0.07315041),
 (906, 0.07259015)]

In [17]:
related = model.similar_items(itemid = 10000)

IndexError: index 10000 is out of bounds for axis 0 with size 1436

In [16]:
related

[(1, 0.06838048),
 (797, 0.0671787),
 (1278, 0.066799164),
 (1282, 0.06669805),
 (283, 0.06660657),
 (1352, 0.06641201),
 (802, 0.06637288),
 (1102, 0.06628359),
 (400, 0.066052005),
 (249, 0.065990135)]

In [ ]:
from ItemItemRecommender import ItemItemRecommender
recommend = ItemItemRecommender()
recommend.fit(data_sparse)


In [127]:
recomenders_for_4th_Disciple = recommend.pred_one_user(11)

In [137]:
ind = np.argwhere(recomenders_for_4th_Disciple)
data.columns[ind]

Index(['It's a New Day', 'Change the Beat (Female Version)', 'Rubber Jam',
       'Makin' Believe That It's You', 'In the Rain', 'Midnight Theme',
       'The Epic of Bitter Triumph', 'N.T.',
       'Love Theme From "Romeo and Juliet" (A Time for Us)', 'Do It Again',
       ...
       'Think Big', 'The Eye of a Needle', 'One Less Bell to Answer',
       'Don't Say Goodnight (It's Time for Love) (Parts 1 & 2)',
       'The Tragedy (Don't Do It)', 'Ain't No Half Steppin'',
       'Kool It (Here Comes the Fuzz)', 'Trilogy of Terror', 'L Dopa',
       'Do It Your Way'],
      dtype='object', length=1011)

In [ ]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit

# Create a numeric user_id and artist_id column
data['user'] = data['user'].astype("category")
data['artist'] = data['artist'].astype("category")
data['user_id'] = data['user'].cat.codes
data['artist_id'] = data['artist'].cat.codes

# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['plays'].astype(float), (data['artist_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['plays'].astype(float), (data['user_id'], data['artist_id'])))

# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)


#---------------------
# FIND SIMILAR ITEMS
#---------------------

# Find the 10 most similar to Jay-Z
item_id = 147068 #Jay-Z
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(item_id, n_similar)

# Print the names of our most similar artists
for item in similar:
    idx, score = item
    print data.artist.loc[data.artist_id == idx].iloc[0]

    
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 2025
user_id = 2025

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item)

artists = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
    artists.append(data.artist.loc[data.artist_id == idx].iloc[0])
    scores.append(score)

# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'artist': artists, 'score': scores})

print recommendations
